# kahiii - 카카오 딥러닝 기반 형태소 분석기

* 소개: https://brunch.co.kr/@kakao-it/308
* 코드: https://github.com/kakao/khaiii

In [1]:
import pandas as pd
import khaiii
api = khaiii.KhaiiiApi()
api.open()

In [50]:
df = pd.read_csv('noun_pairs.csv')
df.head()

,title,okt_pairs,hannanum_pairs,komoran_pairs,mecab_pairs
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?,"[['소비자보호', '실태', '평가', '결과'], ['확인'], ['수']]","[['소비자보호실태평가', '결과'], ['어디'], ['확인'], ['수']]","[['소비자보호', '실태', '평가', '결과'], ['확인'], ['수']]","[['소비자', '보호', '실태', '평가', '결과'], ['확인'], ['수']]"
1,소비자보호실태평가의 평가방법이 궁금합니다.,"[['소비자보호', '실태', '평가'], ['평가', '방법']]","[['소비자보호실태평가'], ['평가방법'], ['궁금']]","[['소비자보호', '실태', '평가'], ['평가', '방법']]","[['소비자', '보호', '실태', '평가'], ['평가', '방법']]"
2,소비자보호실태평가는 어떤 제도인가요?,"[['소비자보호', '실태', '평가'], ['제도']]","[['소비자보호실태평'], ['제도']]","[['소비자보호', '실태', '평가'], ['제도']]","[['소비자', '보호', '실태', '평가'], ['제도']]"
3,금융감독원의 민원대상 금융회사가 궁금합니다.,"[['금융감독원'], ['민원', '대상', '금융', '회사']]","[['금융감독원'], ['민원대상', '금융회사'], ['궁금']]","[['금융감독원'], ['민원', '대상', '금융', '회사']]","[['금융', '감독원'], ['민원', '대상', '금융', '회사']]"
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?,"[['내', '금융', '계좌'], ['한번'], ['조회'], ['수'], ['사...","[['내', '금융계좌'], ['한번'], ['조회'], ['수'], ['사이트']]","[['금융', '계좌'], ['한번'], ['조회'], ['수'], ['사이트']]","[['금융', '계좌'], ['조회'], ['수'], ['사이트']]"


In [47]:
results = pd.DataFrame(df['title'].copy())
results.head()

,title
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?
1,소비자보호실태평가의 평가방법이 궁금합니다.
2,소비자보호실태평가는 어떤 제도인가요?
3,금융감독원의 민원대상 금융회사가 궁금합니다.
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?


In [71]:
def khaiii_tokenize(text):
    morph_list = []
    for word in api.analyze(text):
        for m in word.morphs:
            morph_list.append((m.lex, m.tag))
    return morph_list

In [72]:
results['tokenized'] = results['title'].apply(lambda x: khaiii_tokenize(x))
results.head()

,title,tokenized
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?,"[(소비자, NNG), (보호실, NNG), (태평가, NNG), (결과, NNG)..."
1,소비자보호실태평가의 평가방법이 궁금합니다.,"[(소비자, NNG), (보호실태, NNG), (평가, NNG), (의, JKG),..."
2,소비자보호실태평가는 어떤 제도인가요?,"[(소비자, NNG), (보호실태평가, NNG), (는, JX), (어떤, MM),..."
3,금융감독원의 민원대상 금융회사가 궁금합니다.,"[(금융, NNG), (감독원, NNG), (의, JKG), (민, NNP), (원..."
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?,"[(나, NP), (의, JKG), (금융, NNG), (계좌, NNG), (를, ..."


In [73]:
tokenized_text = results['tokenized'][0]
tokenized_text

[('소비자', 'NNG'),
 ('보호실', 'NNG'),
 ('태평가', 'NNG'),
 ('결과', 'NNG'),
 ('는', 'JX'),
 ('어디', 'NP'),
 ('서', 'JKB'),
 ('확인', 'NNG'),
 ('하', 'XSV'),
 ('ㄹ', 'ETM'),
 ('수', 'NNB'),
 ('있', 'VV'),
 ('나요', 'EF'),
 ('?', 'SF')]

In [92]:
def get_noun_pairs(tokenized_text, noun):
    pairs = []
    token_list = []
    for token in tokenized_text:
        if(token[1] in noun):
            token_list.append(token[0])
        else:
            if (not token_list):
                continue
            pairs.append(token_list)
            token_list = []
    if(token_list):
        pairs.append(token_list)
    return pairs

In [93]:
results['khaiii_pairs'] = results['tokenized'].apply(lambda x: get_noun_pairs(x, ['NNG', 'NNP']))
results

,title,tokenized,khaiii_pairs
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?,"[(소비자, NNG), (보호실, NNG), (태평가, NNG), (결과, NNG)...","[[소비자, 보호실, 태평가, 결과], [확인]]"
1,소비자보호실태평가의 평가방법이 궁금합니다.,"[(소비자, NNG), (보호실태, NNG), (평가, NNG), (의, JKG),...","[[소비자, 보호실태, 평가], [평가, 방법]]"
2,소비자보호실태평가는 어떤 제도인가요?,"[(소비자, NNG), (보호실태평가, NNG), (는, JX), (어떤, MM),...","[[소비자, 보호실태평가], [제도]]"
3,금융감독원의 민원대상 금융회사가 궁금합니다.,"[(금융, NNG), (감독원, NNG), (의, JKG), (민, NNP), (원...","[[금융, 감독원], [민, 원, 대상, 금융, 회사]]"
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?,"[(나, NP), (의, JKG), (금융, NNG), (계좌, NNG), (를, ...","[[금융, 계좌], [조회], [사이트]]"
5,어려운 금융용어 등을 쉽게 찾아볼 수 있는 사이트는 없나요?,"[(어렵, VA), (ㄴ, ETM), (금융, NNG), (용어, NNG), (등,...","[[금융, 용어], [사이트]]"
6,정기예금 및 적금 금리를 한눈에 비교해 볼 수 있는 곳은 없나요?,"[(정기, NNG), (예금, NNG), (및, MAG), (적금, NNG), (금...","[[정기, 예금], [적금, 금리], [한눈], [비교], [곳]]"
7,「보이는 ARS」는 어떻게 이용할 수 있나요?,"[(「, SS), (보이, VV), (는, ETM), (ARS, SL), (」, S...",[[이용]]
8,자동차사고 발생시 양당사자의 과실비율은 어떻게 산정하나요?,"[(자동차사고, NNG), (발생, NNG), (시, NNB), (양당, NNG),...","[[자동차사고, 발생], [양당, 사자], [과실, 비율], [산정]]"
9,"교통사고 시 SUV, RV, 승합차의 렌트비 인정기준은 어떻게 되나요?","[(교통사고, NNG), (시, NNG), (SUV, SL), (,, SP), (R...","[[교통사고, 시], [승합차], [렌트비, 인정, 기준]]"


In [94]:
df['khaiii_pairs'] = results['khaiii_pairs']
df.head()

,title,okt_pairs,hannanum_pairs,komoran_pairs,mecab_pairs,khaiii_pairs
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?,"[['소비자보호', '실태', '평가', '결과'], ['확인'], ['수']]","[['소비자보호실태평가', '결과'], ['어디'], ['확인'], ['수']]","[['소비자보호', '실태', '평가', '결과'], ['확인'], ['수']]","[['소비자', '보호', '실태', '평가', '결과'], ['확인'], ['수']]","[[소비자, 보호실, 태평가, 결과], [확인]]"
1,소비자보호실태평가의 평가방법이 궁금합니다.,"[['소비자보호', '실태', '평가'], ['평가', '방법']]","[['소비자보호실태평가'], ['평가방법'], ['궁금']]","[['소비자보호', '실태', '평가'], ['평가', '방법']]","[['소비자', '보호', '실태', '평가'], ['평가', '방법']]","[[소비자, 보호실태, 평가], [평가, 방법]]"
2,소비자보호실태평가는 어떤 제도인가요?,"[['소비자보호', '실태', '평가'], ['제도']]","[['소비자보호실태평'], ['제도']]","[['소비자보호', '실태', '평가'], ['제도']]","[['소비자', '보호', '실태', '평가'], ['제도']]","[[소비자, 보호실태평가], [제도]]"
3,금융감독원의 민원대상 금융회사가 궁금합니다.,"[['금융감독원'], ['민원', '대상', '금융', '회사']]","[['금융감독원'], ['민원대상', '금융회사'], ['궁금']]","[['금융감독원'], ['민원', '대상', '금융', '회사']]","[['금융', '감독원'], ['민원', '대상', '금융', '회사']]","[[금융, 감독원], [민, 원, 대상, 금융, 회사]]"
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?,"[['내', '금융', '계좌'], ['한번'], ['조회'], ['수'], ['사...","[['내', '금융계좌'], ['한번'], ['조회'], ['수'], ['사이트']]","[['금융', '계좌'], ['한번'], ['조회'], ['수'], ['사이트']]","[['금융', '계좌'], ['조회'], ['수'], ['사이트']]","[[금융, 계좌], [조회], [사이트]]"


In [95]:
df.to_csv('noun_pairs_final.csv', index=False, encoding='utf-8-sig')